In [19]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import yaml

#export
class AppConfig():
    def __init__(self, file_name):
        with open(file_name) as f:
            self.config = yaml.safe_load(f)
        self.database_user = self.config["database_user"]
        self.database_password = self.config["database_password"]
        self.database_name = self.config["database_name"]
        self.database_host = self.config["database_host"]
        self.database_port = self.config["database_port"]
        self.table_index_values_name = self.config["table_index_values_name"]
        self.table_base_rates_name = self.config["table_base_rates_name"]
        self.data_file_folder = self.config["data_file_folder"]
        
class DataFileContainer():
    def __init__(self, file_name):
        with open(file_name) as f:
            self.yaml_file = yaml.safe_load(f)
        self.index_value_files = self.yaml_file["index_data"]
        self.base_rate_files = self.yaml_file["base_rate_data"]

In [21]:
#hide
from nbdev.showdoc import *
from IPython.display import Markdown,display
#import nb2ltx.AppConfig as AppConfig
import pyorient
import pandas as pd
import os.path

In [22]:
config = AppConfig("config.yml")

In [23]:
client = pyorient.OrientDB(config.database_host, config.database_port)
client.connect(config.database_user, config.database_password)

20

In [24]:
if client.db_exists(config.database_name):
   # Drop Database
   client.db_drop(config.database_name)
    
# Create Database
client.db_create(
  config.database_name,
  pyorient.DB_TYPE_DOCUMENT,
  pyorient.STORAGE_TYPE_PLOCAL 
)

database = client.db_open(config.database_name, config.database_user, config.database_password)

In [25]:
client.command("CREATE CLASS " + config.table_index_values_name + " IF NOT EXISTS")
client.command("CREATE CLASS " + config.table_base_rates_name + " IF NOT EXISTS")

[12]

In [26]:
def prepare_index_data(df, isin, region):
    df.columns = df.columns.str.replace(' ','_')
    df.dropna(inplace = True)
    df["ISIN"] = isin
    df["Region"] = region
    
def insert_index_data(index_value_file_list):
    current_index = 1
    for file in index_value_file_list:
        df = pd.read_csv(os.path.join(config.data_file_folder, file["file_name"]))
        prepare_index_data(df, file["isin"], file["region"])
        print("Inserting '" + file["file_name"] + "' (" + str(current_index) + " of " + str(len(index_value_file_list)) + ") ...")
        current_index = current_index + 1
        for index, row in df.iterrows():
            insert_command = "INSERT INTO " + config.table_index_values_name + " CONTENT " + row.to_json()
            client.command(insert_command)

In [44]:
def prepare_base_rate_data(df, institution, region):
    df.columns = df.columns.str.replace(' ','_')
    df.dropna(inplace = True, axis=1, how='all')
    df["Institution"] = institution
    df["Region"] = region
    
def insert_base_rate_data(base_rate_file_list):
    current_index = 1
    for file in base_rate_file_list:
        df = pd.read_excel(os.path.join(config.data_file_folder, file["file_name"]), file["excel_sheet_id"])
        prepare_base_rate_data(df, file["institution"], file["region"])
        print("Inserting '" + file["file_name"] + "' (" + str(current_index) + " of " + str(len(base_rate_file_list)) + ") ...")
        current_index = current_index + 1
        for index, row in df.iterrows():
            insert_command = "INSERT INTO " + config.table_base_rates_name + " CONTENT " + row.to_json()
            client.command(insert_command)

In [ ]:
data_file_container = DataFileContainer("files.yml")

In [28]:
print("-- Inserting index values")
insert_index_data(data_file_container.index_value_files)
print("-- Done!")

Inserting index values
Inserting 'Index CAC40 (FCHI) Entwicklung.csv' (1 of 7) ...
Inserting 'Index DAX Entwicklung.csv' (2 of 7) ...
Inserting 'Index Dow Jones (DJI) Entwicklung.csv' (3 of 7) ...
Inserting 'Index Euronext 100 (N100) Entwicklung.csv' (4 of 7) ...
Inserting 'Index NASDAQ100 (NDX) Entwicklung.csv' (5 of 7) ...
Inserting 'Index S&P500 Entwicklung.csv' (6 of 7) ...
Inserting 'Index STOXX50E Entwicklung seit 2007.csv' (7 of 7) ...
Inserting base_rate values
Inserting 'Statista EZB Leitzins Entwicklung.xlsx' (1 of 2) ...
Inserting 'Statista FED Leitzins Entwicklung.xlsx' (2 of 2) ...
Done!


In [43]:
print("-- Inserting base_rate values")
insert_base_rate_data(data_file_container.base_rate_files)
print("-- Done")

-- Inserting base_rate values
tsdfsdf


,Date_Since,Base_Rate,Institution,Region
0,1999-01-01,3.00,EZB,EU
1,1999-01-04,3.00,EZB,EU
2,1999-01-22,3.00,EZB,EU
3,1999-04-09,2.50,EZB,EU
4,1999-11-05,3.00,EZB,EU
5,2000-02-04,3.25,EZB,EU
6,2000-03-17,3.50,EZB,EU
7,2000-04-28,3.75,EZB,EU
8,2000-06-09,4.25,EZB,EU
9,2000-06-28,4.25,EZB,EU


Inserting 'Statista EZB Leitzins Entwicklung.xlsx' (1 of 2) ...


,Date_Since,Base_Rate,Institution,Region
0,2001-01-03,6.00,FED,USA
1,2001-01-31,5.50,FED,USA
2,2001-03-20,5.00,FED,USA
3,2001-04-18,4.50,FED,USA
4,2001-05-15,4.00,FED,USA
5,2001-06-27,3.75,FED,USA
6,2001-08-21,3.50,FED,USA
7,2001-09-17,3.00,FED,USA
8,2001-10-02,2.50,FED,USA
9,2001-11-06,2.00,FED,USA


Inserting 'Statista FED Leitzins Entwicklung.xlsx' (2 of 2) ...
-- Done


In [46]:
print(client.query("SELECT FROM " + config.table_index_values_name, 10)[0])
print(client.query("SELECT FROM " + config.table_base_rates_name, 10)[0])

{'@index_value':{'Date': '1990-03-01', 'Open': 1836.0, 'High': 1838.0, 'Low': 1827.0, 'Close': 1832.0, 'Adj_Close': 1832.0, 'Volume': 0.0, 'ISIN': 'FR0003500008', 'Region': 'EU'},'version':1,'rid':'#25:0'}
{'@base_rate':{'Date_Since': 915148800000, 'Base_Rate': 3.0, 'Institution': 'EZB', 'Region': 'EU'},'version':1,'rid':'#33:0'}
